In [5]:
import sys
pl_path = "/mnt/c/Users/mm851/PycharmProjects/ProteinLearning"
if pl_path not in sys.path:
    sys.path.append(pl_path)

In [18]:
from pyrosetta import *
import numpy as np
from einops import repeat
import torch
init()
atoms = "N CA C".split(" ")
seq = "ARNDCQEGHILKMFPSTWYV"
pose = pose_from_sequence(seq)
b,n,a = 2,len(seq),len(atoms)
coords = torch.zeros(n,a,3)
for i in range(len(seq)):
    for a_idx,a in enumerate(atoms):
        crds = np.array(list(pose.residue(i+1).atom(a).xyz()))
        coords[i,a_idx] = torch.tensor(crds)
b,n,a = 1,len(seq),len(atoms)
coords = repeat(coords, "n a c -> b n a c",b=b)                            

PyRosetta-4 2020 [Rosetta PyRosetta4.Release.python38.linux 2020.28+release.8ecab77aa50ac1301efe53641e07e09ac91fee3b 2020-07-07T16:41:06] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python38.linux r260 2020.28+release.8ecab77 8ecab77aa50ac1301efe53641e07e09ac91fee3b http://www.pyrosetta.org 2020-07-07T16:41:06
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.8/dist-packages/pyrosetta-2020.28+release.8ecab77-py3.8-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1715512232 seed_offset=0 real_seed=1715512232
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1715512232 RG_type=mt19937


In [19]:
from protein_learning.networks.ipa.rigid import Rigids
bb_rigids = Rigids.RididFromBackbone(coords)

In [23]:
from einops import repeat, rearrange
coords_rep = repeat(coords, "b n a c -> b m (n a) c", m=coords.shape[1])
rrep = bb_rigids.apply(coords_rep)
coords_resh = rearrange(coords,"b n a c -> b () (n a) c")
rresh = bb_rigids.apply(coords_resh)
print(torch.norm(rrep-rresh))
rresh.shape

tensor(0.)


torch.Size([1, 20, 60, 3])